In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['auto', 'scale'],
    'C': [10, 100, 1000, 2000, 3000],
    'decision_function_shape': ['ovo', 'ovr']
}

grid = GridSearchCV(
    estimator=SVC(probability=True),
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=5,
    verbose=3,
    n_jobs=-1
)

grid.fit(X_train, y_train)

# Predictions
grid_predictions = grid.predict(X_test)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


In [8]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 

In [9]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [10]:
print(cm)

[[80  5]
 [ 7 42]]


In [27]:
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [12]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [13]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 100, 'decision_function_shape': 'ovo', 'gamma': 'auto', 'kernel': 'rbf'}: 0.9100355779243318


In [14]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9539015606242497

In [15]:
table=pd.DataFrame.from_dict(re)

In [16]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_decision_function_shape,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014011,0.000959,0.003785,0.000612,10,ovo,auto,linear,"{'C': 10, 'decision_function_shape': 'ovo', 'g...",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,41
1,0.012228,0.002816,0.006418,0.002202,10,ovo,auto,poly,"{'C': 10, 'decision_function_shape': 'ovo', 'g...",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,21
2,0.010909,0.001322,0.003402,0.000335,10,ovo,auto,rbf,"{'C': 10, 'decision_function_shape': 'ovo', 'g...",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,5
3,0.009334,0.000701,0.003701,0.000421,10,ovo,auto,sigmoid,"{'C': 10, 'decision_function_shape': 'ovo', 'g...",0.762677,0.738916,0.655795,0.796284,0.766556,0.744045,0.047743,77
4,0.013311,0.001250,0.003974,0.001114,10,ovo,scale,linear,"{'C': 10, 'decision_function_shape': 'ovo', 'g...",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.064546,0.025281,0.055098,0.036109,3000,ovr,auto,sigmoid,"{'C': 3000, 'decision_function_shape': 'ovr', ...",0.791752,0.714931,0.698113,0.796284,0.766556,0.753527,0.040048,71
76,4.970848,2.425384,0.015311,0.008915,3000,ovr,scale,linear,"{'C': 3000, 'decision_function_shape': 'ovr', ...",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,41
77,5.232206,4.572474,0.023748,0.023770,3000,ovr,scale,poly,"{'C': 3000, 'decision_function_shape': 'ovr', ...",0.822092,0.765553,0.766556,0.924528,0.901744,0.836095,0.066541,29
78,0.327702,0.075460,0.010135,0.009981,3000,ovr,scale,rbf,"{'C': 3000, 'decision_function_shape': 'ovr', ...",0.826263,0.866968,0.851527,0.847020,0.883278,0.855011,0.019206,17


In [21]:
age_input=float(input("'Age:"))
bmi_input=float(input("EstimatedSalary:"))
children_input=float(input("Gender_Male:"))

'Age: 25
EstimatedSalary: 53333
Gender_Male: 1


In [25]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
